<a href="https://colab.research.google.com/github/jankovicsandras/plpgsql_bm25/blob/main/plpgsql_bm25_comparison_20250403.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparison of the following BM25 search libraries:

 - https://github.com/jankovicsandras/plpgsql_bm25 - BM25 search implemented in PL/pgSQL
 - https://github.com/jankovicsandras/bm25opt - faster BM25 search algorithms in Python
 - https://github.com/paradedb/paradedb - Postgres for Search and Analytics
 - https://github.com/xhluca/bm25s - BM25S is an ultrafast implementation of BM25 in pure Python
 - https://github.com/dorianbrown/rank_bm25 - A Collection of BM25 Algorithms in Python

More information on the different BM25 algorithms:

 - https://github.com/dorianbrown/rank_bm25/issues/43#issuecomment-2657715463
 - Kamphuis et al: Which BM25 Do You Mean? A Large-Scale Reproducibility Study of Scoring Variants https://cs.uwaterloo.ca/~jimmylin/publications/Kamphuis_etal_ECIR2020_preprint.pdf


## installing PostgreSQL

In [1]:
! sudo apt install gnupg2 wget nano
! sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
! curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/postgresql.gpg
! sudo apt update
! sudo apt install postgresql-17 postgresql-contrib-17 postgresql-server-dev-17
! service postgresql start
! sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
! sudo -u postgres psql -c "ALTER SYSTEM SET paradedb.pg_search_telemetry TO 'off';"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
gnupg2 is already the newest version (2.2.27-3ubuntu2.1).
Suggested packages:
  hunspell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 280 kB of archives.
After this operation, 881 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 nano amd64 6.2-1ubuntu0.1 [280 kB]
Fetched 280 kB in 1s (536 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-p

## installing ParadeDB pg_search

In [2]:
! export PARADEDB_TELEMETRY=false
! lsb_release -a
! sudo apt-get install -y libicu70

# https://github.com/paradedb/paradedb/releases
! wget -nc https://github.com/paradedb/paradedb/releases/download/v0.15.6/postgresql-17-pg-search_0.15.6-1PARADEDB-jammy_amd64.deb
! dpkg -i postgresql-17-pg-search_0.15.6-1PARADEDB-jammy_amd64.deb

# https://docs.paradedb.com/deploy/pg_search
# https://github.com/paradedb/paradedb/tree/dev/pg_search

! echo "shared_preload_libraries = 'pg_search'" >> /etc/postgresql/17/main/postgresql.conf
! service postgresql stop
! sleep 10
! service postgresql start
! sudo -u postgres psql -c "CREATE EXTENSION pg_search;"
! sudo -u postgres psql -c "ALTER SYSTEM SET paradedb.pg_search_telemetry TO 'off';"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libicu70 is already the newest version (70.1-2).
libicu70 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
--2025-04-03 09:24:41--  https://github.com/paradedb/paradedb/releases/download/v0.15.6/postgresql-17-pg-search_0.15.6-1PARADEDB-jammy_amd64.deb
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/660776503/44f8c709-fc99-465f-9c67-d50ba32cf0cc?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250403%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250403T092441Z&X-Amz-Expires=300&X-Amz-Signature=361105fdee7

## init psycopg2 helper and misc. functions

In [3]:
! pip install psycopg2

import psycopg2

! rm mlog.txt

# current time string
def nowf() :
  return datetime.datetime.now().isoformat().replace(":","-")


# write to logfile and print
def mlog( *args ) :
  s = ''
  try :
    for arg in args :
      s += str(arg)+' '
  except Exception as ex :
    print( '!!!! mlog() ERROR ', ex, args )

  try :
    with open('mlog.txt','a+') as f :
      f.write( '\n' + s )
  except Exception as ex :
    with open('mlog.txt','w+') as f :
      f.write( '\n' + s )
  print( s )


# execute SQL
def msq( t, verbose=False ) :
  res = []
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      try :
        cur.execute(t)
        res = cur.fetchall()
        if verbose :
          theader = '|'.join( [ str(cdesc[0]) for cdesc in cur.description ] )
          mlog(theader)
          for r in res:
            trow =  '|'.join( [ str(c) for c in r ] )
            mlog(trow)
      except Exception as ex :
        mlog('!!!! msq() ERROR ',ex,'|',t,'|')
  return res


# execute SQL
def msq2( t, s, verbose=False ) :
  res = []
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      try :
        cur.execute(t, (s,))
        res = cur.fetchall()
        if verbose :
          theader = '|'.join( [ str(cdesc[0]) for cdesc in cur.description ] )
          mlog(theader)
          for r in res:
            trow =  '|'.join( [ str(c) for c in r ] )
            mlog(trow)
      except Exception as ex :
        mlog('!!!! msq() ERROR ',ex,'|',t,'|')
  return res


# check if correct is in top k results
def iscorrectintopk(correct, res) :
  cr = correct.replace('"','\'')
  if len(res) < 1 : return 0
  for cell in res[0] :
    if str(cell).replace('"','\'') == cr : return 2
  for ri in range(1,len(res)) :
    for cell in res[ri] :
      if str(cell).replace('"','\'') == cr : return 1
  return 0


# trigram functions ############################################################

# word to trigram list
def word2tri( w ) :
  termchar = '¤'
  w2 = w.strip().lower()
  if len(w2) < 2 : return []
  if len(w2) == 2 : return [ w2+termchar ]
  if len(w2) == 3 : return [ w2 ]
  if len(w2) == 4 : return [ w2[0:3], w2[1:4] ] # TODO: remove this? 1450/2000
  if len(w2) > 4 : return [ w2[0:3], w2[1:4], w2[2:5] ] # TODO: remove w2[2:5] ?


# text to trigram list
def text2tris( t ) :
  wa = t.split()
  res = []
  for w in wa :
    res += word2tri( w )
  return res



rm: cannot remove 'mlog.txt': No such file or directory


## Test datasets

In [4]:

# downloading datasets

! mkdir datasets

! cd datasets && wget -nc https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
! cd datasets && wget -nc https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/queries.jsonl

! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/msmarco_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/robust04_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/nq_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/trec_covid_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/fiqa_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/dbpedia_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/scidocs_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/scifacts_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/arguana_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/bioasq_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/climate_fever_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/cqadupstack_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/fever_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/hotpotqa_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/nfcorpus_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/quora_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/signal_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/touche_synthetic_queries_100k.jsonl
! cd datasets && wget -nc https://zav-public.s3.amazonaws.com/inpars/trec_news_synthetic_queries_100k.jsonl

import random, json

# parameters
samplenum = 50
columnname = 'doc_text'
idcolumnname = 'id'

# Wordpress dataset

# load from jsonl
wcorpus = []
with open('datasets/corpus.jsonl') as f:
  wcstr = f.read()
  wcorpus = wcstr.split('\n')
mlog('len(wcorpus)',len(wcorpus))

# create sampled corpus, items, questions
sampledwcorpus = random.sample( wcorpus, samplenum )
items = []
qqs = []
for i in range(0,len(sampledwcorpus)) :
  wjs = json.loads(sampledwcorpus[i])
  items.append( { columnname: wjs['text'] } )
  qqs.append( [ wjs['title'], items[-1][columnname] ] )

# questions and solutions
random.shuffle(qqs)
wpquestions = [ q[0] for q in qqs ]
wpquestionsolutions = [ q[1] for q in qqs ]

################################################################################

# Other datasets

datasetnames = ['primitive','wordpress','msmarco', 'robust04', 'nq', 'trec_covid', 'fiqa', 'dbpedia', 'scidocs', 'scifacts', 'arguana', 'bioasq', 'climate_fever', 'cqadupstack', 'fever', 'hotpotqa', 'nfcorpus', 'quora', 'signal', 'touche', 'trec_news']
fnpostfix = '_synthetic_queries_100k.jsonl'

# data and question containers
alldata = {}
questions = {}
questionsolutions = {}
embeddedquestions = {}

for dsname in datasetnames : #[:2]
  print('\n---- ',dsname)
  if dsname == 'primitive' :
    alldata[dsname] = [ { columnname:'one' }, { columnname:'one two two' }, { columnname:'one two two three three three' }, { columnname:'four four one' }, { columnname:'five two' }, ]
    questions[dsname] = [ 'one one two', 'one two two', 'one three two three', 'three four four', 'five' ]
    questionsolutions[dsname] = [ 'one', 'two two', 'three three three', 'four four one', 'five two' ]
  elif dsname == 'wordpress' :
    alldata[dsname] = items
    questions[dsname] = wpquestions
    questionsolutions[dsname] = wpquestionsolutions
  else :
    # load
    with open( 'datasets/'+dsname+fnpostfix ) as f:
      alldata[dsname] = [json.loads(line) for line in f]
    # shuffle and truncate to samplenum
    random.shuffle(alldata[dsname])
    alldata[dsname] = alldata[dsname][:samplenum]
    # display
    #for i in range(0,10) :
    #  print(alldata[dsname][i])

    # suffling again and creating questions and questionsolutions
    questions[dsname] = []
    questionsolutions[dsname] = []
    random.shuffle(alldata[dsname])
    for i in range( 0, len(alldata[dsname]) ) :
      questions[dsname].append( alldata[dsname][i]['question'] )
      questionsolutions[dsname].append( alldata[dsname][i][columnname] )

  # export to CSV for Postgres
  csvfilename = dsname+'_items.csv'
  with open( 'datasets/'+csvfilename, 'w+' ) as f:
    f.write( 'id;'+columnname+'\n' )
    for i in range( 0, len(alldata[dsname]) ) :
      f.write( str(i+1)+';"' + alldata[dsname][i][columnname].replace('"','\'') + '"\n' )
  print( 'datasets/'+csvfilename, 'is created' )

  # Postgres creating items table by importing from CSV
  tablename = dsname + '_items_table'
  msq('DROP TABLE IF EXISTS '+tablename+' CASCADE;')
  msq('CREATE TABLE '+tablename+' ( id SERIAL, '+columnname+' TEXT );')
  msq('COPY '+tablename+' FROM \'/content/datasets/'+csvfilename+'\' DELIMITER \';\' CSV HEADER;')
  msq('SELECT * FROM '+tablename+';')


--2025-04-03 09:25:01--  https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.90, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e3/a1/e3a12a6c68820b63bcd7ae09aa898026eba332004f38239070204b3b146060bc/089b11077372513eca8cc16653485aff1f232f8be18d8c6263be4b3b2bda0078?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27corpus.jsonl%3B+filename%3D%22corpus.jsonl%22%3B&Expires=1743675902&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzY3NTkwMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UzL2ExL2UzYTEyYTZjNjg4MjBiNjNiY2Q3YWUwOWFhODk4MDI2ZWJhMzMyMDA0ZjM4MjM5MDcwMjA0YjNiMTQ2MDYwYmMvMDg5YjExMDc3MzcyNTEzZWNhOGNjMTY2NTM0ODVhZmYxZjIzMmY4YmUxOGQ4YzYyNjNiZTRiM2IyYmRhMDA3OD9yZXNwb25zZS1jb25

## installing plpgsql_bm25, BM25opt, BM25s, rank_bm25

In [5]:

# downloading and installing plpgsql_bm25.sql
! wget -nc https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql
! psql postgresql://@/postgres -f plpgsql_bm25.sql


# downloading BM25opt
! wget -nc https://raw.githubusercontent.com/jankovicsandras/bm25opt/refs/heads/main/BM25opt.py


# https://github.com/xhluca/bm25s/tree/main
# Install all extra dependencies
! pip install bm25s[full]
# If you want to use stemming for better results, you can install a stemmer
! pip install PyStemmer
# To speed up the top-k selection process, you can install `jax`
! pip install jax[cpu]


# install rank_bm25 and imports
! pip install rank_bm25


--2025-04-03 09:27:23--  https://raw.githubusercontent.com/jankovicsandras/plpgsql_bm25/refs/heads/main/plpgsql_bm25.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45187 (44K) [text/plain]
Saving to: ‘plpgsql_bm25.sql’

plpgsql_bm25.sql    100%[===================>]  44.13K  --.-KB/s    in 0.007s  

2025-04-03 09:27:23 (5.83 MB/s) - ‘plpgsql_bm25.sql’ saved [45187/45187]

psql:plpgsql_bm25.sql:66: NOTICE:  function stopwordfilter() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:125: NOTICE:  function bm25simpletokenize() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:146: NOTICE:  function count_words_in_array() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:167: N

## running the comparsion of plpgsql_bm25, BM25opt, ParadeDB pg_search, bm25s, rank_bm25

In [6]:

! rm mlog.txt

from BM25opt import BM25opt, tokenizer_default
from rank_bm25 import BM25Okapi, BM25L, BM25Plus
import json, time, math, subprocess
import pandas as pd
import bm25s
import Stemmer


def bm25s_tokenize_debug( o ) :
  inv_map = { v: k for k, v in o.vocab.items() }
  s = ' '.join( [ inv_map[wid] for wid in o.ids[0] ] )
  return s


# parameters
k = 5 # top k result limit
verbose = False
stopwordslanguage = 'en'
plpgsql_bm25_algos = [ '', 'l', 'plus', 'robertson', 'luceneaccurate', 'atire', 'bm25l', 'bm25plus' ]
all_algos = ['paradedbbm25','bm25s','rank_bm25','bm25opt','tribm25opt']
for bm25algo in plpgsql_bm25_algos :
  all_algos.append('plpgsql_bm25_'+bm25algo)

# english stopwords
stopwords_en = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "would", "should", "could", "ought", "i''m", "you''re", "he''s", "she''s", "it''s", "we''re", "they''re", "i''ve", "you''ve", "we''ve", "they''ve", "i''d", "you''d", "he''d", "she''d", "we''d", "they''d", "i''ll", "you''ll", "he''ll", "she''ll", "we''ll", "they''ll", "isn''t", "aren''t", "wasn''t", "weren''t", "hasn''t", "haven''t", "hadn''t", "doesn''t", "don''t", "didn''t", "won''t", "wouldn''t", "shan''t", "shouldn''t", "can''t", "cannot", "couldn''t", "mustn''t", "let''s", "that''s", "who''s", "what''s", "here''s", "there''s", "when''s", "where''s", "why''s", "how''s", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very"]


# results and stats
searchresults = {}
stats = {}


# dataset loop
for dsname in datasetnames : #[:2]

  # tablename
  tablename = dsname + '_items_table'

  # results and stats
  searchresults[ dsname+'_paradedbbm25' ] = []
  searchresults[ dsname+'_bm25s' ] = []
  searchresults[ dsname+'_rank_bm25' ] = []
  searchresults[ dsname+'_bm25opt' ] = []
  searchresults[ dsname+'_tribm25opt' ] = []

  stats[ dsname+'_paradedbbm25_correct' ] = []
  stats[ dsname+'_bm25s_correct' ] = []
  stats[ dsname+'_rank_bm25_correct' ] = []
  stats[ dsname+'_bm25opt_correct' ] = []
  stats[ dsname+'_tribm25opt_correct' ] = []

  stats[ dsname+'_paradedbbm25_speed' ] = []
  stats[ dsname+'_bm25s_speed' ] = []
  stats[ dsname+'_rank_bm25_speed' ] = []
  stats[ dsname+'_bm25opt_speed' ] = []
  stats[ dsname+'_tribm25opt_speed' ] = []

  # creating indexes

  # pgpgsql_bm25 all algorithms ------------------------------------------------
  # Postgres has already "items" table, tokenization is built-in
  for bm25algo in plpgsql_bm25_algos :
    rkey = dsname+'_plpgsql_bm25_'+bm25algo
    searchresults[ rkey ] = []
    stats[ rkey+'_correct' ] = []
    stats[ rkey+'_speed' ] = []
    starttime = time.time()
    msq( 'SELECT bm25createindex( \''+tablename+'\', \''+columnname+'\', \''+bm25algo+'\', \''+stopwordslanguage+'\'  );', verbose )
    createindexspeed = time.time()-starttime
    stats[ rkey+'_createindexspeed' ] = createindexspeed
    mlog( 'plpgsql_bm25 createindex', tablename, columnname, bm25algo, stopwordslanguage, 'took', createindexspeed, 's.' )

  # rank_bm25 ------------------------------------------------------------------
  # preparing tokenized corpus
  corpus = [ item[columnname] for item in alldata[dsname] ]
  starttime = time.time()
  tokenized_corpus = [ tokenizer_default( item[columnname] ) for item in alldata[dsname] ]
  rank_bm25_index = BM25Plus( tokenized_corpus )
  createindexspeed = time.time()-starttime
  stats[ dsname+'_rank_bm25_createindexspeed' ] = createindexspeed
  mlog( 'rank_bm25 BM25Plus', tablename, 'took', createindexspeed, 's.' )

  # BM25opt --------------------------------------------------------------------
  starttime = time.time()
  bm25opt_index = BM25opt( corpus, algo='plus' )
  createindexspeed = time.time()-starttime
  stats[ dsname+'_bm25opt_createindexspeed' ] = createindexspeed
  mlog( 'BM25opt', tablename, 'took', createindexspeed, 's.' )

  # ParadeDB pg_search ---------------------------------------------------------
  msq('DROP SCHEMA IF EXISTS '+dsname+'_paradedbbm25'+' CASCADE;')
  starttime = time.time()
  msq('CREATE INDEX '+dsname+'_paradedbbm25'+' ON '+tablename+' USING bm25 (id, '+columnname+') WITH (key_field=\'id\');')
  createindexspeed = time.time()-starttime
  stats[ dsname+'_paradedbbm25_createindexspeed' ] = createindexspeed
  mlog( 'paradedbbm25 CREATE INDEX', tablename, 'took', createindexspeed, 's.' )

  # BM25s ----------------------------------------------------------------------
  starttime = time.time()
  #bm25s_stemmer = Stemmer.Stemmer( "english" )
  bm25s_corpus_tokens = bm25s.tokenize( corpus, stopwords="en" ) #, stemmer=bm25s_stemmer )
  bm25s_retriever = bm25s.BM25( method="lucene", idf_method="lucene" )
  bm25s_retriever.index( bm25s_corpus_tokens )
  createindexspeed = time.time()-starttime
  stats[ dsname+'_bm25s_createindexspeed' ] = createindexspeed
  mlog( 'bm25s index', tablename, 'took', createindexspeed, 's.' )

  # trigram BM25opt ------------------------------------------------------------
  starttime = time.time()
  tribm25opt_index = BM25opt( [ text2tris(text) for text in corpus ], algo='okapi', tokenizer_function=lambda x : x )
  createindexspeed = time.time()-starttime
  stats[ dsname+'_tribm25opt_createindexspeed' ] = createindexspeed
  mlog( 'tribm25opt', tablename, 'took', createindexspeed, 's.' )

  ##############################################################################

  # Running the questions
  for qi,q in enumerate( questions[dsname] ) :

    # tokenize and print question ----------------------------------------------
    tokenizedquestion = tokenizer_default( q )
    if verbose :
      mlog('\n----Question',qi,':',q,' | Tokenized: ',tokenizedquestion ) #,' | tri ',text2tris( q ))
      if questionsolutions[dsname] and qi < len( questionsolutions[dsname] ) : print( 'Solution:', questionsolutions[dsname][qi] )

    # plpgsql_bm25 BM25 search -------------------------------------------------
    for bm25algo in plpgsql_bm25_algos :
      rkey = dsname+'_plpgsql_bm25_'+bm25algo
      if verbose : mlog('|',rkey,'|')
      pcmdstr = 'SELECT * FROM bm25topk( \''+tablename+'\', \''+columnname+'\', %s, '+str(k)+', \''+bm25algo+'\', \''+stopwordslanguage+'\' );'
      starttime = time.time()
      searchresults[ rkey ].append( msq2( pcmdstr, q, verbose ) )
      stats[ rkey+'_speed' ].append( (time.time()-starttime) )
      stats[ rkey+'_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ rkey ][-1] ) )

    # ParadeDB BM25 search -----------------------------------------------------
    if verbose : mlog('ParadeDB|')
    pcmdstr = 'SELECT id, paradedb.score(id) AS score, '+columnname+' AS doc FROM '+tablename+' WHERE '+columnname+' @@@ paradedb.match(\''+columnname+'\', %s ) ORDER BY score DESC LIMIT '+str(k)+';' #  USING bm25
    starttime = time.time()
    searchresults[ dsname+'_paradedbbm25' ].append( msq2( pcmdstr, q, verbose ) )
    stats[ dsname+'_paradedbbm25_speed' ].append( (time.time()-starttime) )
    stats[ dsname+'_paradedbbm25_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ dsname+'_paradedbbm25' ][-1] ) )

    # bm25s search -------------------------------------------------------------
    # Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k).
    searchresults[ dsname+'_bm25s' ].append( [] )
    starttime = time.time()
    bm25s_question_tokens = bm25s.tokenize( q, stopwords="en" ) #, stemmer=bm25s_stemmer )
    bm25s_results, bm25s_scores = bm25s_retriever.retrieve( bm25s_question_tokens, k=min(k, len(corpus)) )
    stats[ dsname+'_bm25s_speed' ].append( (time.time()-starttime) )
    for i in range( bm25s_results.shape[1] ):
      searchresults[ dsname+'_bm25s' ][-1].append( [ bm25s_results[0, i]+1, bm25s_scores[0, i], alldata[dsname][ bm25s_results[0, i] ][columnname]  ] )  # database id starts with 1
    stats[ dsname+'_bm25s_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ dsname+'_bm25s' ][-1] ) )
    if verbose :
      mlog('bm25s   |')
      for br in searchresults[ dsname+'_bm25s' ][-1] : mlog( br[0], '|', br[1], '|', br[2] )

    # rank_bm25 BM25 search ----------------------------------------------------
    starttime = time.time()
    doc_scores = rank_bm25_index.get_scores( tokenizedquestion )
    stats[ dsname+'_rank_bm25_speed' ].append( (time.time()-starttime) )
    # top k
    searchresults[ dsname+'_rank_bm25' ].append( [ [i,s] for i,s in enumerate(doc_scores) ] )
    searchresults[ dsname+'_rank_bm25' ][-1].sort( key=lambda x:x[1], reverse=True )
    searchresults[ dsname+'_rank_bm25' ][-1] = searchresults[ dsname+'_rank_bm25' ][-1][:k]
    for i in range( 0, len(searchresults[ dsname+'_rank_bm25' ][-1]) ) :
      searchresults[ dsname+'_rank_bm25' ][-1][i].append( alldata[dsname][ searchresults[ dsname+'_rank_bm25' ][-1][i][0] ][columnname] )
      searchresults[ dsname+'_rank_bm25' ][-1][i][0] += 1 # database id starts with 1
    stats[ dsname+'_rank_bm25_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ dsname+'_rank_bm25' ][-1] ) )
    #if verbose :
    #  mlog('rank_bm25  |')
    #  for br in searchresults['rank_bm25'][-1]  : mlog( br[0], '|', br[1], '|', br[2] )

    # bm25opt BM25 search ------------------------------------------------------
    starttime = time.time()
    searchresults[ dsname+'_bm25opt' ].append( bm25opt_index.topk( q, k ) )
    stats[ dsname+'_bm25opt_speed' ].append( (time.time()-starttime) )
    for i in range( 0, len( searchresults[ dsname+'_bm25opt'][-1] ) ) :
      searchresults[ dsname+'_bm25opt' ][-1][i].append( alldata[dsname][ searchresults[ dsname+'_bm25opt'][-1][i][0] ][columnname] )
      searchresults[ dsname+'_bm25opt' ][-1][i][0] += 1 # database id starts with 1
    stats[ dsname+'_bm25opt_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ dsname+'_bm25opt' ][-1] ) )
    if verbose :
      mlog('bm25opt   |')
      for br in searchresults[ dsname+'_bm25opt' ][-1] : mlog( br[0], '|', br[1], '|', br[2] )

    # trigram bm25opt BM25 search ----------------------------------------------
    starttime = time.time()
    searchresults[ dsname+'_tribm25opt' ].append( tribm25opt_index.topk( text2tris( q ), k ) )
    stats[ dsname+'_tribm25opt_speed' ].append( (time.time()-starttime) )
    for i in range( 0, len( searchresults[ dsname+'_tribm25opt' ][-1] ) ) :
      searchresults[ dsname+'_tribm25opt' ][-1][i].append( alldata[dsname][ searchresults[ dsname+'_tribm25opt' ][-1][i][0] ][columnname] )
      searchresults[ dsname+'_tribm25opt' ][-1][i][0] += 1 # database id starts with 1
    stats[ dsname+'_tribm25opt_correct' ].append( iscorrectintopk( questionsolutions[dsname][qi], searchresults[ dsname+'_tribm25opt' ][-1] ) )
    #if verbose :
    #  mlog('tribm25opt|')
    #  for br in searchresults['tribm25opt'][-1] : mlog( br[0], '|', br[1], '|', br[2] )

    ## End of questions loop
  ## End of dataset loop



plpgsql_bm25 createindex primitive_items_table doc_text  en took 0.06447267532348633 s. 
plpgsql_bm25 createindex primitive_items_table doc_text l en took 0.03728318214416504 s. 
plpgsql_bm25 createindex primitive_items_table doc_text plus en took 0.12622761726379395 s. 
plpgsql_bm25 createindex primitive_items_table doc_text robertson en took 0.07757997512817383 s. 
plpgsql_bm25 createindex primitive_items_table doc_text luceneaccurate en took 0.16161656379699707 s. 
plpgsql_bm25 createindex primitive_items_table doc_text atire en took 0.1760413646697998 s. 
plpgsql_bm25 createindex primitive_items_table doc_text bm25l en took 0.16434240341186523 s. 
plpgsql_bm25 createindex primitive_items_table doc_text bm25plus en took 0.23090195655822754 s. 
rank_bm25 BM25Plus primitive_items_table took 0.00013184547424316406 s. 
BM25opt primitive_items_table took 0.00016808509826660156 s. 
!!!! msq() ERROR  no results to fetch | DROP SCHEMA IF EXISTS primitive_paradedbbm25 CASCADE; | 
!!!! msq() 

Split strings:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/5 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/5 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# processing and displaying results. See stats_corrects.csv for total correctness statistics.

! rm stats_corrects.csv

stats2 = {}
for statname in stats :
  if type(stats[statname]) == list :
    stats2[statname+'_sum'] = sum(stats[statname])
    mlog( statname+'_sum', stats2[statname+'_sum'] )

cells = []

csvfilename = 'stats_corrects.csv'
with open( csvfilename, 'w+' ) as f:
  f.write( '#;'+';'.join(all_algos)+'\n' )
  for dsname in datasetnames :
    cells.append( [] )
    s = dsname+';'
    for algo in all_algos :
      s += str(stats2[ dsname+'_'+algo+'_correct_sum' ]) + ';'
      cells[-1].append( stats2[ dsname+'_'+algo+'_correct_sum' ] )
    s = s[:-1] + '\n'
    f.write( s )
  cellstransposed = [list(i) for i in zip(*cells)]
  sums = [ str(sum( c )) for c in cellstransposed ]
  s = 'sum;'+';'.join(sums)+'\n'
  f.write( s )

print( csvfilename, 'is created' )


rm: cannot remove 'stats_corrects.csv': No such file or directory
primitive_paradedbbm25_correct_sum 5 
primitive_bm25s_correct_sum 5 
primitive_rank_bm25_correct_sum 5 
primitive_bm25opt_correct_sum 5 
primitive_tribm25opt_correct_sum 5 
primitive_paradedbbm25_speed_sum 0.23139715194702148 
primitive_bm25s_speed_sum 0.6879885196685791 
primitive_rank_bm25_speed_sum 0.0008392333984375 
primitive_bm25opt_speed_sum 0.00021219253540039062 
primitive_tribm25opt_speed_sum 0.00017905235290527344 
primitive_plpgsql_bm25__correct_sum 5 
primitive_plpgsql_bm25__speed_sum 0.3097035884857178 
primitive_plpgsql_bm25_l_correct_sum 5 
primitive_plpgsql_bm25_l_speed_sum 0.31162214279174805 
primitive_plpgsql_bm25_plus_correct_sum 5 
primitive_plpgsql_bm25_plus_speed_sum 0.2648766040802002 
primitive_plpgsql_bm25_robertson_correct_sum 5 
primitive_plpgsql_bm25_robertson_speed_sum 0.42450976371765137 
primitive_plpgsql_bm25_luceneaccurate_correct_sum 5 
primitive_plpgsql_bm25_luceneaccurate_speed_sum 0